In [ ]:
import os
os.getpid()

## Importing module

This could be better with a module init or other import setup.

In [ ]:
import inspect
import hybridcuda
cures = hybridcuda.initcuda()
hybridcuda.registerheader("hybpython.cuh", os.getcwd() + os.sep + ".." + os.sep + ".." + os.sep + "hybpython.cuh")
cures

In [ ]:
class hybridkernel:
    gridDimX = 1
    blockDimX = 1
    shared = 0
    stream = 0
    def __init__(self, func):
        self.hc = hybridcuda.processfunction(func)
        self.hc = hybridcuda.cudajitcode(self.hc)
        self.hc = hybridcuda.ptxlinkcode(self.hc)
        
    def __call__(self, *args):
        self.hc = hybridcuda.launch(self.hc, self.gridDimX,1,1, self.blockDimX,1,1, self.shared,self.stream, *args)

    def __getitem__(self, args):
        if (type(args) != tuple):
            self.grid = args
            return self
        # args is a tuple...
        if (len(args) > 0):
            self.grid = args[0]
        if (len(args) > 1):
            self.block = args[1]
        if (len(args) > 2):
            self.shared = args[2]
        if (len(args) > 3):
            self.stream = args[3]
        return self

#decorator definition
def hybridfunction(func):
    return hybridkernel(func)

In [ ]:
class threadIdx(object):
    __hybrid_cuda__intrinsic_type__ = "threadIdx"
    x : int = 0
    y : int = 0
    z : int = 0
        
class blockIdx(object):
    __hybrid_cuda__intrinsic_type__ = "blockIdx"
    x : int = 0
    y : int = 0
    z : int = 0
        
class blockDim(object):
    __hybrid_cuda__intrinsic_type__ = "blockDim"
    x : int = 1
    y : int = 1
    z : int = 1
        
class gridDim(object):
    __hybrid_cuda__intrinsic_type__ = "gridDim"
    x : int = 1
    y : int = 1
    z : int = 1

## Calling a Function

In [ ]:
def niter(maxiter,x,y):
    z = complex(0,0)
    # back to [-2..2]^2 from a 512*512 image
    c = complex(x - 256.0, y - 256.0) / 128.0
    for n in range(maxiter):
        # early exit when the value is above 2 in norm.
        if ((z.real * z.real + z.imag * z.imag) > 4.0):
            return n
        z = z * z + c
    return maxiter

@hybridkernel
def mandelbrot(maxiter,sizex,sizey,pixels):
    y = threadIdx.y + blockDim.y * blockIdx.y
    while (y < sizey):
        x = threadIdx.x + blockDim.x * blockIdx.x
        while (x < sizex):
            pixels[x+y*sizex] = niter(maxiter,x,y)
            x += blockDim.x * gridDim.x
        y += blockDim.y * gridDim.y

In [ ]:
import numpy
countimage = numpy.zeros(512*512)

In [ ]:
mandelbrot[256,256](32,512,512,countimage)

## Coloring and Display

In [ ]:
def falsecolor(a,max_iter):
    if (a == max_iter):
        # dark red illustrates the inside of the set
        return (48,0,0)
    red = int(127.0 * float(a) / float(max_iter))
    green = int(200.0 * float(a) / float(max_iter))
    blue = int(90.0 * float(a) / float(max_iter))
    return (red,green,blue) ;

In [ ]:
from matplotlib.pyplot import imshow
from PIL import Image

img = Image.new('RGB', (512,512), "black")
pixels = img.load()

In [ ]:
def applycolor(sizex,sizey,countimage,pixels):
    for x in range(sizex):
        for y in range(sizey):
            pixels[x,y] = falsecolor(countimage[sizex*y+x],32)

applycolor(512,512,countimage,pixels)

In [ ]:
%matplotlib inline

In [ ]:
img

## Going with a lambda

(ongoing work - unsupported yet)

In [ ]:
@hybridkernel
def mandelbrot_lambda(func,sizex,sizey,pixels):
    y = threadIdx.y + blockDim.y * blockIdx.y
    while (y < sizey):
        x = threadIdx.x + blockDim.x * blockIdx.x
        while (x < sizex):
            pixels[x+y*sizex] = func(x,y)
            x += blockDim.x * gridDim.x
        y += blockDim.y * gridDim.y

In [ ]:
def niter(maxiter,x,y):
    z = complex(0,0)
    # back to [-2..2]^2 from a 512*512 image
    c = complex(x - 256.0, y - 256.0) / 128.0
    for n in range(maxiter):
        # early exit when the value is above 2 in norm.
        if ((z.real * z.real + z.imag * z.imag) > 4.0):
            return n
        z = z * z + c
    return maxiter

myfunc = lambda x,y: niter(32, x,y)

countimage2 = numpy.zeros(512*512)
mandelbrot_lambda[256,256](myfunc, 512,512,countimage2)


In [ ]:
applycolor(512,512,countimage2,pixels)

In [ ]:
img